In [1]:
# importing packages
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
%matplotlib inline

In [2]:
path = 'catsvsdogs\\images\\'
size = 150

training_images = [] 
test_images = []
training_labels = []
test_labels = []

def make_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

make_directory('dataset\\train\\cats')
make_directory('dataset\\train\\dogs')
make_directory('dataset\\test\\cats')
make_directory('dataset\\test\\dogs')
make_directory('npz-data')

for pic in os.listdir(path):
    if pic.split('.')[0]=='cat':
        if int(pic.split('.')[1])<=1000:
            img = cv2.imread(path+pic)
            img = cv2.resize(img,(size,size),interpolation = cv2.INTER_AREA)
            training_images.append(img)
            training_labels.append(0)
            cv2.imwrite('dataset\\train\\cats\\cats{}.jpg'.format(pic.split('.')[1]) , img)
        else:
            img = cv2.imread(path+pic)
            img = cv2.resize(img,(size,size),interpolation = cv2.INTER_AREA)
            test_images.append(img)
            test_labels.append(0)
            cv2.imwrite('dataset\\test\\cats\\cats{}.jpg'.format(pic.split('.')[1]) , img)
            
    elif pic.split('.')[0]=='dog':
        if int(pic.split('.')[1])<=1000:
            img = cv2.imread(path+pic)
            img = cv2.resize(img,(size,size),interpolation = cv2.INTER_AREA)
            training_images.append(img)
            training_labels.append(1)
            cv2.imwrite('dataset\\train\\dogs\\dogs{}.jpg'.format(pic.split('.')[1]) , img)
        else:
            img = cv2.imread(path+pic)
            img = cv2.resize(img,(size,size),interpolation = cv2.INTER_AREA)
            test_images.append(img)
            test_labels.append(1)
            cv2.imwrite('dataset\\test\\dogs\\dogs{}.jpg'.format(pic.split('.')[1]) , img)
            

np.savez('npz-data\\training_images',training_images)
np.savez('npz-data\\test_images',test_images)
np.savez('npz-data\\training_labels',training_labels)
np.savez('npz-data\\test_labels',test_labels)

print('training and test data saved !!!')
print(len(training_images))
print(len(training_labels))
print(len(test_images))
print(len(test_labels))

training and test data saved !!!
2002
2002
1000
1000


In [3]:
def load_training_and_test_data():
    train_X = np.load('npz-data\\training_images.npz')['arr_0']
    train_y = np.load('npz-data\\training_labels.npz')['arr_0']
    test_X = np.load('npz-data\\test_images.npz')['arr_0']
    test_y = np.load('npz-data\\test_labels.npz')['arr_0']
    
    return train_X,train_y,test_X,test_y

In [4]:
# Just checking our data
for i in range (1,11):
    random_index = np.random.randint(0,len(training_images))
    cv2.imshow('image '+str(random_index),training_images[random_index])
    if training_labels[random_index]==1:
        print('dog')
    else:
        print('cat')
    cv2.waitKey(0)
cv2.destroyAllWindows()

dog
dog
cat
dog
dog
cat
cat
dog
cat
dog


In [5]:
train_X,train_y,test_X,test_y = load_training_and_test_data()

# Converting (20002,) to (2002,1)
train_y = train_y.reshape(train_y.shape[0],1)
test_y = test_y.reshape(test_y.shape[0],1)

#Normalizing our data in 0 to 1 range
train_X  = train_X.astype('float32')
test_X   = test_X.astype('float32')
train_X /= 255
test_X  /= 255

print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

(2002, 150, 150, 3)
(2002, 1)
(1000, 150, 150, 3)
(1000, 1)


In [6]:
# CNN model
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten,Dropout,MaxPooling2D

input_shape = (train_X.shape[1],train_X.shape[2],train_X.shape[3])

model =Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss=['binary_crossentropy'],
             optimizer='rmsprop',
             metrics=['accuracy'])

print(model.summary())

Using TensorFlow backend.
W1113 11:51:26.092553 14836 deprecation_wrapper.py:119] From C:\Users\sharma ji\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1113 11:51:26.244696 14836 deprecation.py:323] From C:\Users\sharma ji\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36992)            

In [7]:
batch_size = 16
epochs = 25

classifier = model.fit( train_X,
                        train_y,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_data=(test_X,test_y),
                        shuffle=True)



W1113 11:51:33.782557 14836 deprecation_wrapper.py:119] From C:\Users\sharma ji\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 2002 samples, validate on 1000 samples
Epoch 1/25
2002/2002 [==============================] - 50s 25ms/step - loss: 0.7191 - accuracy: 0.5215 - val_loss: 0.6694 - val_accuracy: 0.5920
Epoch 2/25
2002/2002 [==============================] - 49s 24ms/step - loss: 0.6555 - accuracy: 0.6419 - val_loss: 0.7104 - val_accuracy: 0.5580
Epoch 3/25
2002/2002 [==============================] - 45s 23ms/step - loss: 0.5957 - accuracy: 0.6713 - val_loss: 0.5764 - val_accuracy: 0.6820
Epoch 4/25
2002/2002 [==============================] - 41s 21ms/step - loss: 0.5538 - accuracy: 0.7358 - val_loss: 0.5748 - val_accuracy: 0.6970
Epoch 5/25
2002/2002 [==============================] - 44s 22ms/step - loss: 0.4965 - accuracy: 0.7677 - val_loss: 0.5707 - val_accuracy: 0.7070
Epoch 6/25
2002/2002 [==============================] - 40s 20ms/step - loss: 0.4495 - accuracy: 0.7882 - val_loss: 0.6855 - val_accuracy: 0.6450
Epoch 7/25
2002/2002 [==============================] - 39s 20ms/step - loss

OSError: Unable to create file (unable to open file: name = 'model\cats_vs_dogs_classifier_V1.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [9]:
model.save('model\\cats_vs_dogs_classifier_V1.h5')

test_loss,test_accuracy = model.evaluate(test_X,test_y,verbose=1)
print('Test Loss ---> ',test_loss)
print('Test Accuray ---> ',test_accuracy*100)

1000/1000 [==============================] - 4s 4ms/step
Test Loss --->  3.5332479276657103
Test Accuray --->  71.39999866485596


In [38]:
# Testing our model
from keras.models import load_model
classifier = load_model('model\\cats_vs_dogs_classifier_V1.h5')
BLACK=[0,0,0]
acc=[]
for i in range(0,10):
    random_index = np.random.randint(0,len(test_X))
    test_img = test_X[random_index]
    test_large = cv2.resize(test_img,None,fx=2,fy=2,interpolation=cv2.INTER_CUBIC)
    test_img = test_img.reshape(1,150,150,3)
    res = (classifier.predict_classes(test_img,1,verbose=0)[0])
    acc.append(np.equal(res[0],test_labels[random_index]))
    
    if res==[0]:
        res='cat'
    else :
        res='dog'
    
    test_large = cv2.copyMakeBorder(test_large,0,0,0,200,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(test_large,text=res,org=(300,105),fontScale=2,thickness=2,fontFace=cv2.FONT_HERSHEY_TRIPLEX,color=(0,255,0))
    cv2.imshow('predictions',test_large)
    cv2.waitKey(0)
cv2.destroyAllWindows()
print('accuracy is ---> ',round(np.mean(acc)*100,2))

accuracy is --->  100.0


# DATA AUGMENTATION

In [43]:
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array

data_generator = ImageDataGenerator(rotation_range=40,
                                             height_shift_range=0.2,
                                             width_shift_range=0.2,
                                             horizontal_flip=True,
                                             shear_range=0.2,
                                             fill_mode='nearest',
                                             zoom_range=0.2)
img = load_img('dog.jpg')
img = img_to_array(img)
img = img.reshape((1,)+img.shape)
i=0
for batch in data_generator.flow(img,save_to_dir='output',save_prefix='dog',save_format='jpeg'):
    i+=1
    if i>35:
        break

In [46]:
input_shape = (150, 150, 3)
img_width = 150
img_height = 150

nb_train_samples = 2000
nb_validation_samples = 1000
batch_size = 16
epochs = 25


train_data_dir = 'dataset/train'
test_data_dir  = 'dataset/test'

train_data_generator = ImageDataGenerator(   rescale = 1. / 255,
                                             rotation_range=40,
                                             height_shift_range=0.2,
                                             width_shift_range=0.2,
                                             horizontal_flip=True,
                                             fill_mode='nearest',
                                             zoom_range=0.2)

test_data_generator = ImageDataGenerator(rescale = 1. / 255)

train_generator = train_data_generator.flow_from_directory(train_data_dir,
                                                          target_size = (img_width,img_height), 
                                                          batch_size = batch_size,
                                                          class_mode = 'binary',
                                                           shuffle = True)

test_generator = test_data_generator.flow_from_directory(test_data_dir,
                                                          target_size = (img_width,img_height), 
                                                          batch_size = batch_size,
                                                          class_mode = 'binary',
                                                           shuffle = False)



Found 2002 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# CREATING MODEL SAME AS WE DID ABOVE

In [51]:
# Creating out model
from keras.layers import Activation 
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

print(model.summary())

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_11 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
activation_12 (Activation)   (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 34, 34, 128)      

In [52]:
nb_train_samples = 2000
nb_validation_samples = 1000

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    validation_data = test_generator,
    validation_steps = nb_validation_samples // batch_size)



Epoch 1/25
125/125 [==============================] - 56s 448ms/step - loss: 0.7195 - accuracy: 0.5015 - val_loss: 0.6925 - val_accuracy: 0.5554
Epoch 2/25
125/125 [==============================] - 51s 405ms/step - loss: 0.6961 - accuracy: 0.5337 - val_loss: 0.5341 - val_accuracy: 0.5528
Epoch 3/25
125/125 [==============================] - 48s 382ms/step - loss: 0.6946 - accuracy: 0.5876 - val_loss: 0.4868 - val_accuracy: 0.5854
Epoch 4/25
125/125 [==============================] - 47s 379ms/step - loss: 0.6718 - accuracy: 0.6153 - val_loss: 0.6025 - val_accuracy: 0.6545
Epoch 5/25
125/125 [==============================] - 47s 377ms/step - loss: 0.6594 - accuracy: 0.6274 - val_loss: 0.6472 - val_accuracy: 0.6535
Epoch 6/25
125/125 [==============================] - 47s 377ms/step - loss: 0.6461 - accuracy: 0.6380 - val_loss: 0.8735 - val_accuracy: 0.6433
Epoch 7/25
125/125 [==============================] - 47s 376ms/step - loss: 0.6409 - accuracy: 0.6581 - val_loss: 0.6092 - val_ac

In [57]:
model.save('model\\cats_vs_dogs_classifier_V2_augmented.h5')

test_loss,test_accuracy = model.evaluate(test_X,test_y,verbose=1)
print('Test Loss ---> ',test_loss)
print('Test Accuracy ---> ',test_accuracy*100)

1000/1000 [==============================] - 5s 5ms/step
Test Loss --->  0.7308197417259217
Test Accuracy --->  63.999998569488525


In [56]:
# Plotting our loss charts
import matplotlib.pyplot as plt

history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, val_loss_values,linewidth=2.0, marker = '+', markersize=10.0, label='Validation/Test Loss')
plt.plot(epochs, loss_values, linewidth=2.0, marker = '4', markersize=10.0, label='Training Loss')

plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

AttributeError: 'Sequential' object has no attribute 'history'

In [ ]:
# Plotting our accuracy charts

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, val_acc_values, linewidth=2.0, marker = '+', markersize=10.0, label='Validation/Test Accuracy')
plt.plot(epochs, acc_values,  linewidth=2.0, marker = '4', markersize=10.0 ,label='Training Accuracy')

plt.xlabel('Epochs') 
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()